### XPath и CSS-селекторы

Селекторы помогают найти элемент на странице для дальнейшего взаимодействия (клик, ввод текста и т.д.) - языки запросов

XPath — это язык запросов для навигации по XML/HTML-документам и выборки узлов (элементов, атрибутов, текста).
* Используется путь к нужному элементу, подобно структуре файловой системы
* Легко выбирает элементы на основе иерархии, текста или условий



CSS-селекторы — это синтаксис, используемый в CSS для стилизации элементов, но его также можно применять для выборки элементов в парсерах и инструментах автоматизации.

### Блок кода для парсинга

In [3]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import os
import time
# glob - библиотека для поиска файлов и директорий по заданному шаблону 
import glob     

# Настройка параметров загрузки
download_dir = r"C:\Users\Катерина Андреевна\code\парсинг\pdf + supplementary information"
if not os.path.exists(download_dir):
    os.makedirs(download_dir)
    print(f"Создана папка: {download_dir}")

# Конфигурация ChromeOptions
options = uc.ChromeOptions()
prefs = {
    "download.default_directory": download_dir,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "plugins.always_open_pdf_externally": True,
    "profile.default_content_settings.popups": 0,
    "profile.content_settings.exceptions.automatic_downloads.*.setting": 1  # Разрешаем авто-загрузки
}
options.add_experimental_option("prefs", prefs)
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--disable-extensions")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

# Инициализация драйвера
driver = uc.Chrome(options=options)
wait = WebDriverWait(driver,30)

# Функция для скачивания файлов с разным расширением (для скачивание статьи и supplementary)
def wait_for_download(download_dir, extension, timeout=60):        # timeout - максимально время ожидания, сек
    print("Ожидание загрузки файла...")
    # выводит сообщение на экран, информируя о начале загрузки файла
    start_time = time.time()        # фиксирует начальное время отсчета - время начала ожидания 
    while time.time() - start_time < timeout:       # цикл будет выполняться до тех пор, разница между текущим временем и временем начала не превысит заданный timeout
        files = glob.glob(os.path.join(download_dir, f"*.{extension}"))      # ищет все файлы с указанным расширением
        if files:
            # Найдем самый свежий файл
            files.sort(key=os.path.getmtime, reverse=True)
            print(f"Файл статьи загружен: {files[0]}")
            return True     # true как сигнал об успешном обнаружении файла
        time.sleep(1)       # приостановка выполнения на 1 секунду перед следующей проверкой
    return False

# Открываем страницу
try:
    print("Открываем страницу статьи")
    driver.get('https://doi.org/10.1007/s12161-021-02044-x')
    
    # Даем странице полностью загрузиться
    time.sleep(5)

    # Запускаем скачивание pdf - клик по кнопке 
    try:
        # Вариант 1: По классу ссылки pdf-download__link
        pdf_button = wait.until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, 'a.c-pdf-download__link'))
        )
    except:
        try:
            # Вариант 2: По атрибуту title
            pdf_button = wait.until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, 'a[data-test="pdf-link"]'))
            )
        except:
            # Вариант 3: По XPath с учетом структуры
            pdf_button = wait.until(
                EC.element_to_be_clickable((By.XPATH, '//a[.//span[contains(text(), "Download PDF")]]'))
                )
    
    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", pdf_button)        # для прокрутки страницы до кнопки
    time.sleep(1)
    
    print("Нажатие кнопки Download PDF")

    # Пробуем разные способы клика
    try:
        pdf_button.click()
    except:
        driver.execute_script("arguments[0].click();", pdf_button)
    print("Кнопка Download PDF нажата")

    # Ожидание загрузки файла
    wait_for_download(download_dir, "pdf")
       
        # Скачиваем Supplementary Information 
    try:
        print("Попытка найти ссылку на Supplementary Information")
        esm_link = wait.until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, 'a[data-test="supp-info-link"]'))
        )
        
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", esm_link)
        time.sleep(1)
        
        print("Нажатие ссылки Supplementary Information")
        try:
            esm_link.click()
        except:
            driver.execute_script("arguments[0].click();", esm_link)
        print("Ссылка Supplementary Information нажата")
        
        # Определяем расширение файла из ссылки
        href = esm_link.get_attribute("href")
        extension = href.split('.')[-1].lower()  # получаем расширение из URL
        
        # Ожидание загрузки ESM файла
        wait_for_download(download_dir, extension)
        
    except Exception as e:
        print(f"Не удалось скачать Supplementary Information: {str(e)}")

finally:
    # Закрываем все вкладки и драйвер
    if len(driver.window_handles) > 0:
        for handle in driver.window_handles:
            driver.switch_to.window(handle)
            driver.close()
    driver.quit()
    print("Браузер закрыт")

print(f"Проверьте папку загрузки: {download_dir}")

Открываем страницу статьи
Нажатие кнопки Download PDF
Кнопка Download PDF нажата
Ожидание загрузки файла...
Файл статьи загружен: C:\Users\Катерина Андреевна\code\парсинг\pdf + supplementary information\EFSA Journal - 2019 -  - Safety evaluation of the food enzyme alpha‐amylase from a genetically modified Bacillus subtilis .pdf
Попытка найти ссылку на Supplementary Information
Нажатие ссылки Supplementary Information
Ссылка Supplementary Information нажата
Ожидание загрузки файла...
Файл статьи загружен: C:\Users\Катерина Андреевна\code\парсинг\pdf + supplementary information\12161_2021_2044_MOESM1_ESM.docx
Браузер закрыт
Проверьте папку загрузки: C:\Users\Катерина Андреевна\code\парсинг\pdf + supplementary information
